# MCP Server Isolation Template

This notebook demonstrates the production MCP server template with:
- `create_mcp_server()` with `AuthSessionMiddleware`
- Workspace path translation
- `@context_result()` for large results
- `@tool_permission` decorators
- `PrivateData` compliance flagging
- Error classification: `ToolRetryError` vs `ToolFatalError`
- Client-side isolation switching via `MCPServerPrivateData`
- MCP server-to-client log messages via `ctx.info()`

Start two instances of the template server (normal and isolated):
```bash
fastmcp run agentic_patterns/mcp/template/server.py:mcp --transport http --port 8000
fastmcp run agentic_patterns/mcp/template/server.py:mcp --transport http --port 8001
```

The client connects to both. Before private data is loaded, calls go to port 8000. After `load_sensitive_dataset` flags the session, the client switches to port 8001 (the isolated instance). In production these would be separate containers with different network policies.

In [ ]:
import logging

# force=True is needed in Jupyter (root logger already has handlers, basicConfig is a no-op without it)
logging.basicConfig(
    level=logging.WARNING, format="%(name)s %(levelname)s: %(message)s", force=True
)
logging.getLogger("agentic_patterns.core.mcp").setLevel(logging.INFO)

from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.mcp import get_mcp_client
from agentic_patterns.core.compliance.private_data import session_has_private_data

In [ ]:
# Reset private data flag from previous runs (deletes the .private_data file)
from agentic_patterns.core.compliance.private_data import PrivateData

pd = PrivateData()
pd.has_private_data = False

## Connect to the template server

`get_mcp_client()` reads `config.yaml` and returns `MCPServerStrict` (single URL) or `MCPServerPrivateData` (when `url_isolated` is configured).

The client is pre-wired with a `log_handler` that forwards MCP server log messages (`ctx.info()`, `ctx.warning()`, etc.) to Python logging.

In [ ]:
server = get_mcp_client("template")
print(type(server).__name__)

In [ ]:
agent = get_agent(toolsets=[server])

## Normal tool call: workspace file read

The `read_file` tool uses `@tool_permission(READ)`, `@context_result()`, and `read_from_workspace()`. The agent sees `/workspace/...` paths, never host paths.

Watch the log output -- the server emits `ctx.info("Reading file: ...")` which arrives on the client via the MCP `notifications/message` mechanism.

In [ ]:
print(f"Private data: {session_has_private_data()}")
async with agent:
    result, _ = await run_agent(
        agent, "Write 'hello world' to /workspace/hello.txt, then read it back."
    )
    print(result.result.output)

## Retryable error: ToolRetryError

The `search_records` tool raises `ToolRetryError` for empty queries. FastMCP converts this to a `ToolError`, which PydanticAI surfaces as `ModelRetry` -- the LLM gets the error message and another chance to provide valid arguments.

We deliberately ask the agent to pass an empty query so the retry fires. Watch the output for:
1. The server's `ctx.warning(...)` showing the empty query was rejected
2. The verbose agent-step log showing a retry followed by a second tool call with a valid query

In [ ]:
print(f"Private data: {session_has_private_data()}")
async with agent:
    result, _ = await run_agent(
        agent,
        "Call the search_records tool with an empty string '' as the query argument.",
        verbose=True,
    )
    print(result.result.output)

## Private data flagging and isolation switch

- Before loading sensitive data, `session_has_private_data()` returns `False` and the MCP client targets the normal server instance. 
- After calling `load_sensitive_dataset`, the tool flags the session via `PrivateData.add_private_dataset()` and emits a `ctx.warning()` log message.
- From that point on, if the client is `MCPServerPrivateData`, all MCP calls route to the isolated server instance.


Watch for two signals in the output:
1. The server's `ctx.warning(...)` message forwarded via the MCP log handler
2. The client's `is_isolated` property flipping from `False` to `True`

In [ ]:
print(f"Private data   : {session_has_private_data()}")
print(
    f"Isolated before: {getattr(server, 'is_isolated', 'N/A (single-instance client)')}"
)

async with agent:
    result, _ = await run_agent(agent, "Load the 'patient_records' sensitive dataset")
    print(result.result.output)

In [ ]:
print(f"Private data  : {session_has_private_data()}")
print(
    f"Isolated after: {getattr(server, 'is_isolated', 'N/A (single-instance client)')}"
)

The session is now marked as **containing private data. **

Any `@tool_permission(CONNECT)` tools would be blocked, and if `url_isolated` were configured, the MCP client would have already switched to the isolated server instance. The switch is a one-way ratchet -- it never reverts within a session.

The log output above should show the server's warning message (`Session flagged as containing private data...`) delivered via the MCP protocol's `notifications/message` mechanism and forwarded to Python logging by the client's `log_handler`.